### Introducción a la Investigación Operativa y la Optimización

### • Polyak, Nesterov y ADAM

**Nazareno Faillace Mullen - Departamento de Matemática, FCEN, UBA**

In [2]:
import numpy as np
from numpy.linalg import norm
from time import time, perf_counter

# Método del gradiente con paso fijo

El Método de Gradiente con paso fijo consiste en fijar $\gamma>0$ e iterar:
$$x_{k+1} = x_k - \gamma \nabla f(x_k)$$

**Teorema**: si $f:\mathbb{R}^n\rightarrow\mathbb{R}$ es acotada inferiormente y $\nabla f$ es Lipschitz, es decir, $\exists L>0$ tal que:
$$\lVert \nabla f(x) - \nabla f(y)\rVert \leq L\lVert x-y\rVert \quad \forall\;x,y\in \mathbb{R}^n$$
entonces el Método de Gradiente con paso fijo converge para $\gamma\in(0, \frac{2}{L})$.

**Teorema**: si $f:\mathbb{R}^n\rightarrow\mathbb{R}$ es convexa, acotada inferiormente y $\nabla f$ es Lipschitz, tomando $\gamma=\frac{1}{L}$, el Método del Gradiente con paso fijo verifica que:
$$f(x_k)-f(x^\star) \leq \frac{L}{2k}\lVert x^\star - x_0\rVert$$


## Funciones cuadrátricas

Consideremos la función cuadrática $f(x)=\frac{1}{2}x^TQx + b^Tx + c$ con $Q$ definida positiva ($Q\succ 0$). Hemos visto que:
$$\nabla f(x) = Qx+b$$

En particular:
$$\nabla f(x) - \nabla f(y) = (Qx+b) - (Qy+b) = Q(x-y) \Rightarrow \lVert \nabla f(x) - \nabla f(y)\rVert \leq \lVert Q \rVert \lVert x-y\rVert$$
entonces ¡$\nabla f$ es Lipschitz con constante $\lVert Q \rVert$!

### Ejercicio

Consideremos la función $f(x)=\frac{1}{2}x^T Q x$ con:
$$Q=\begin{pmatrix}1 & 0 & \dots & \dots & 0 \\
0 & 2 & 0 & \dots & 0 \\
0 & 0 & 3 & \dots & 0 \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
0 & \dots & \dots & 0 & N \\
\end{pmatrix}$$

Como $Q$ es definida positiva, el mínimo es $x^\star = (0,\dots,0)$

1. Implementar la funcion `metodo_gradiente_cuad_fijo` que implemente el Método del Gradiente con paso fijo $\frac{1}{L}$ para funciones cuadráticas. Esta función es análoga a `metodo_gradiente_cuad` que implementamos la clase pasada, pero la longitud del paso debe venir dada como argumento.

In [3]:
def metodo_gradiente_cuad_fijo(A, b, x, paso, eps=1e-5, k_max=1000):
  """
  Aplica el método del gradiente con paso fijo para funciones cuadraticas.
  A : matriz de la funcion cuadratica (np.array)
  b : vector de la funcion cuadratica (np.array)
  x: punto inicial (numpy.array)
  paso: longitud de paso fija (float | int)
  eps: valor de tolerancia para la norma del gradiente (float)
  k_max: limite de iteraciones (int)
  """
  # COMPLETAR
  k = 0
  d = -A@x - b
  while k <= k_max and np.linalg.norm(d) > eps:
    x = x + paso*d
    d = (-A)@x - b
    k += 1
  print(f'Cantidad de iteraciones: {k}')
  return x

2. Aplicar el método para $f(x)=\frac{1}{2}x^T Q x$ con $N=10$ y un máximo de 10000 iteraciones y mostrar el resultado obtenido para cada uno de los siguientes casos:<br>
a) con longitud de paso fija $\frac{1}{\lVert Q \rVert}$<br><br>
b) con longitud de paso fija $\frac{2}{\lVert Q \rVert}$<br><br>
c) con longitud de paso fija $\frac{2.1}{\lVert Q \rVert}$<br><br>
d) con longitud de paso fija $\frac{0.8}{\lVert Q \rVert}$<br><br>
¿Qué se observa?


In [4]:
N = 10
b = np.zeros(N)
x_0 = np.ones(N)
Q = np.diag(np.arange(N)+1)

x = metodo_gradiente_cuad_fijo(Q, b, x_0, 1/N, k_max=10000)
x@Q@x

Cantidad de iteraciones: 110


np.float64(8.577329159116975e-11)

3. Para distintos valores de $N$, comparar la cantidad de iteraciones y tiempo de ejecución del Método del Gradiente con Paso Fijo $\frac{1}{\lVert Q \rVert}$ con:
- el Método de Gradiente para cuadráticas de la clase pasada (`metodo_gradiente_cuad`)
- con el Método de Gradiente para cuadráticas con la mitad del paso óptimo en cada iteración (que llamaremos `metodo_gradiente_cuad_05`

¿Qué observamos que sucede con `metodo_gradiente_cuad_fijo` a medida que aumenta $N$? ¿Por qué te parece que ocurre esto?

In [5]:
def metodo_gradiente_cuad(A, b, x, eps=1e-5, k_max=1000):
  """
  Aplica el método del gradiente.
  func: funcion a optimizar (function)
  x: punto inicial (numpy.array)
  eps: valor de tolerancia para la norma del gradiente (float)
  k_max: limite de iteraciones (int)
  """
  k = 0
  d = -A@x - b
  while k <= k_max and np.linalg.norm(d) > eps:
    t = (d@d)/(d@A@d)
    x = x + t*d
    d = (-A)@x - b
    k += 1
  print(f'Cantidad de iteraciones: {k}')
  return x

def metodo_gradiente_cuad_05(A, b, x, eps=1e-5, k_max=1000):
  """
  Aplica el método del gradiente con mitad del paso optimo.
  func: funcion a optimizar (function)
  x: punto inicial (numpy.array)
  eps: valor de tolerancia para la norma del gradiente (float)
  k_max: limite de iteraciones (int)
  """
  k = 0
  d = -A@x - b
  while k <= k_max and np.linalg.norm(d) > eps:
    t = (d@d)/(d@A@d)*0.5
    x = x + t*d
    d = (-A)@x - b
    k += 1
  print(f'Cantidad de iteraciones: {k}')
  return x


In [6]:
N = 150
b = np.zeros(N)
x_0 = np.ones(N)
Q = np.diag(np.arange(N)+1)

grad_start = perf_counter()
x = metodo_gradiente_cuad(Q, b, x_0, k_max=10000)
print(f'Tiempo del Método del Gradiente: {perf_counter()-grad_start:.2f}')

grad_05_start = perf_counter()
x = metodo_gradiente_cuad_05(Q, b, x_0, k_max=10000)
print(f'Tiempo del Método del Gradiente con mitad de t*: {perf_counter()-grad_05_start:.2f}')

grad_fijo_start = perf_counter()
x = metodo_gradiente_cuad_fijo(Q, b, x_0, 1/N, k_max=10000)
print(f'Tiempo del Método de Gradiente con paso fijo: {perf_counter()-grad_fijo_start:.2f}')

Cantidad de iteraciones: 885
Tiempo del Método del Gradiente: 0.10
Cantidad de iteraciones: 214
Tiempo del Método del Gradiente con mitad de t*: 0.02
Cantidad de iteraciones: 1722
Tiempo del Método de Gradiente con paso fijo: 0.05


# Método de Polyak

En cada iteración, Polyak propone agregar un término de inercia:
$$x_{k+1} = x_k - \alpha \nabla f(x_k) + \beta (x_k-x_{k-1})$$
generalmente con $\beta\in [0,1]$.

Para $f\in C^2$ tal que existen $m,M>0$ tales que para todo $x$ los autovalores de $Hf(x)$ cumplen que $m\leq \lambda \leq M$, Polyak demuestra que los valores óptimos para $\alpha$ y $\beta$ son:
$$\alpha = \frac{4}{(\sqrt{m}+\sqrt{M})^2} \quad \beta=\left(\frac{\sqrt{𝜅} - 1}{\sqrt{𝜅} + 1}\right)^2$$
donde $𝜅=\frac{M}{m}$

## Funciones cuadráticas

En el caso de las funciones cuadráticas, podemos tomar $m$ como el mínimo autovalor de $Q$ y $M$ como el máximo autovalor de $Q$.

Entonces ¿qué representa $𝜅$?

In [7]:
from numpy.linalg import norm

def derivada_parcial(func,x,i):
  """
  Aproxima la i-esima derivada parcial de la función en x, utilizando diferencias centradas.
  func: función a la que se le desea calcular la i-esima derivada parcial (function)
  x: punto en el cual se desea calcular la i-esima derivada parcial (array de numpy)
  i: indice de la coordenada parcial (int)
  """
  h = 0.1
  e_i = np.zeros(x.shape[0])  # np.zeros(len(x))
  e_i[i] = 1
  z = (func(x + h*e_i) - func(x - h*e_i))/(2*h)
  h = h/2
  y = (func(x + h*e_i) - func(x - h*e_i))/(2*h)
  error = norm(y-z)
  eps = 1e-8
  while error>eps and (y != np.nan) and (y != np.inf) and y!= 0:
      error = norm(y-z)
      z = y
      h = h/2
      y = (func(x + h*e_i) - func(x - h*e_i))/(2*h)
  return z

def gradiente(func,x):
  """
  Aproxima el gradiente de la función en x.
  func: función a la que se le desea calcular el gradiente (function)
  x: punto en el cual se desea calcular el gradiente (array de numpy)
  """
  grad = np.empty(x.shape[0]) # np.zeros(x.shape[0])
  for i in range(x.shape[0]):
    grad[i] = derivada_parcial(func, x, i)
  return grad

### Ejercicios

1. Implementar `metodo_polyak_cuad` que lleve a cabo el Método de Polyak para una función cuadrática. La función toma además como argumento los valores de $m$ y $M$.

In [8]:
def metodo_polyak_cuad(A, b, x, m, M, eps=1e-5, k_max=1000):
  """
  Aplica el método de Polyak para funciones cuadraticas.
  A : matriz de la funcion cuadratica (np.array)
  b : vector de la funcion cuadratica (np.array)
  x: punto inicial (numpy.array)
  m: valor que acota inferiormente los autovalores del hessiano (float)
  M: valor que acota superiormente los autovalores del hessiano (float)
  eps: valor de tolerancia para la norma del gradiente (float)
  k_max: limite de iteraciones (int)
  """
  xs = []
  xs.append(x)
  xs.append(x)
  k = 1
  d = -A@x - b
  alpha = 4/((np.sqrt(m)+np.sqrt(M))**2)
  ka = M/m
  beta = ((np.sqrt(ka) - 1)/(np.sqrt(ka) + 1))**2
  while k <= k_max and np.linalg.norm(d) > eps:
    prevx = xs[k - 1]
    xs.append(xs[k] - alpha*(Q@xs[k] + b) + beta*(xs[k] - prevx))
    d = (-A)@xs[k+1] - b
    k += 1
  print(f'Cantidad de iteraciones: {k}')
  return xs[k]

2. Aplicar el método a la función $f(x)=\frac{1}{2}x^TQx$ para $Q$ definida en el ejercicio anterior, con $N=10$.

In [9]:
N = 10
b = np.zeros(N)
x_0 = np.ones(N)
Q = np.diag(np.arange(N)+1)

x = metodo_polyak_cuad(Q, b, x_0, 1, N)
x@Q@x

Cantidad de iteraciones: 28


np.float64(7.83002367398256e-12)

3. Para $N=1500$ y `k_max=10000`, ejecutar la siguiente celda para comparar el tiempo y la cantidad de iteraciones del Método de Polyak con `metodo_gradiente_cuad_05`

In [10]:
N = 1500
b = np.zeros(N)
x_0 = np.ones(N)
Q = np.diag(np.arange(N)+1)

grad_05_start = perf_counter()
x = metodo_gradiente_cuad_05(Q, b, x_0, k_max=10000)
print(f'Tiempo del Método del Gradiente con mitad de t*: {perf_counter()-grad_05_start:.2f}')

polyak_start = perf_counter()
x = metodo_polyak_cuad(Q, b, x_0, 1, N, k_max=10000)
print(f'Tiempo del Método de Polyak: {perf_counter()-polyak_start:.2f}')

Cantidad de iteraciones: 890
Tiempo del Método del Gradiente con mitad de t*: 25.81
Cantidad de iteraciones: 499
Tiempo del Método de Polyak: 14.17


# Método de Nesterov (_Nestorov Accelerated Gradient_)

Nesterov propone modificar el punto donde se calcula el gradiente:
$$x_{k+1} = x_k - \alpha_k \nabla f(x_k+ \beta_k(x_k-x_{k-1})) + \beta (x_k-x_{k-1})$$



Para $f\in C^2$ tal que existen $m,M>0$ tales que para todo $x$ los autovalores de $Hf(x)$ cumplen que $m\leq \lambda \leq M$, Nesterov prueba que es óptimo  fijar $\alpha_k$ y $\beta_k$ en:
$$\alpha = \frac{1}{M} \quad \beta=\frac{\sqrt{M} - \sqrt{m}}{\sqrt{M} + \sqrt{m}}$$

En este caso, se puede probar que:
$$f(x_k)-f(x^\star) \leq \frac{2L}{(k+1)^2}\lVert x^\star - x_0\rVert^2$$

## Ejercicios

1. Implementar la funcion `metodo_nesterov_cuad` que aplique el Método de Nesterov a funciones cuadráticas.

In [20]:
def metodo_nesterov_cuad(A, b, x, m, M, eps=1e-5, k_max=1000):
  """
  Aplica el método de Nesterov para funciones cuadraticas.
  A : matriz de la funcion cuadratica (np.array)
  b : vector de la funcion cuadratica (np.array)
  x: punto inicial (numpy.array)
  m: valor que acota inferiormente los autovalores del hessiano (float)
  M: valor que acota superiormente los autovalores del hessiano (float)
  eps: valor de tolerancia para la norma del gradiente (float)
  k_max: limite de iteraciones (int)
  """
  
  xs = []
  xs.append(x)
  xs.append(x)
  k = 1
  d = -A@x - b
  alpha = 1/M
  beta = ((np.sqrt(M) - np.sqrt(m))/(np.sqrt(M) + np.sqrt(m)))
  while k <= k_max and np.linalg.norm(d) > eps:
    prevx = xs[k - 1]
    xs.append(xs[k] - alpha*(Q@(xs[k]+beta*(xs[k] - prevx))) + beta*(xs[k] - prevx))
    d = (-A)@xs[k+1] - b
    k += 1
  print(f'Cantidad de iteraciones Nesterov: {k}')
  return xs[k]

2. Aplicar el método a la función $f(x)=\frac{1}{2}x^TQx$ para $Q$ definida en el ejercicio anterior, con $N=10$.

In [21]:
N = 10
b = np.zeros(N)
x_0 = np.ones(N)
Q = np.diag(np.arange(N)+1)

x = metodo_nesterov_cuad(Q, b, x_0, 1, N, k_max=100000)
x@Q@x

Cantidad de iteraciones Nesterov: 38


np.float64(9.798392023683198e-11)

3. Para $N=1500$, comparar el tiempo y la cantidad de iteraciones con `metodo_gradiente_cuad` y `metodo_polyak_cuad`

In [22]:
N = 1500
b = np.zeros(N)
x_0 = np.ones(N)
Q = np.diag(np.arange(N)+1)

grad_05_start = perf_counter()
x = metodo_gradiente_cuad_05(Q, b, x_0, k_max=10000)
print(f'Tiempo del Método del Gradiente con mitad de t*: {perf_counter()-grad_05_start:.2f}')

polyak_start = perf_counter()
x = metodo_polyak_cuad(Q, b, x_0, 1, N, k_max=10000)
print(f'Tiempo del Método de Polyak: {perf_counter()-polyak_start:.2f}')

nesterov_start = perf_counter()
x = metodo_nesterov_cuad(Q, b, x_0, 1, N, k_max=10000)
print(f'Tiempo del Método de Nesterov: {perf_counter()-nesterov_start:.2f}')

Cantidad de iteraciones: 890
Tiempo del Método del Gradiente con mitad de t*: 25.41
Cantidad de iteraciones: 499
Tiempo del Método de Polyak: 14.18
Cantidad de iteraciones Nesterov: 549
Tiempo del Método de Nesterov: 15.70


# ADAM

ADAM es uno de los métodos más utilizados actualmente, especialmente para Redes Neuronales. Por un lado, consideramos el **momento** como una media móvil exponencial de los gradientes de iteraciones anteriores:
$$m_{k+1} = \beta_1 m_{k}+(1-\beta_1)\nabla f(x_{k}) \quad \beta_1\in[0,1)$$

Por otro lado, consideramos la media móvil de las magnitudes de los gradientes de iteraciones anteriores:
$$v_{k+1} = \beta_2 v_{k} + (1-\beta_2)\underbrace{\nabla f(x_k)\odot ∇f(x_k)}_{\substack{\text{multiplicación} \\ \text{coordenada a coordenada}}}$$

La actualización del paso en cada iteración se da por:
$$x_{k+1} = x_k - \alpha \frac{m_k}{\sqrt{v_k}+\varepsilon}$$
donde $\varepsilon$ es incluido para estabilidad numérica y evitar dividir por $0$ (podemos tomar $\varepsilon=10^{-8}$).

Como $m_0=v_0=0$, para evitar que los $m_k$ y los $v_k$ estén muy sesgados hacia $0$, en cada iteración se realiza una corrección:
$$\hat{m}_k=\frac{m_k}{(1-\beta_1^k)} \quad \hat{v_k}=\frac{v_t}{(1-\beta_2^k)}$$
y consideraremos:
$$x_{k+1} = x_k - \alpha \frac{\hat{m}_k}{\sqrt{\hat{v}_k}+\varepsilon}$$

Valores usuales de los parámetros: $\alpha < 1,\; \beta_1=0.9,\; \beta_2=0.99$

**Pseudocódigo de ADAM**

adam($f$,$x$, $\alpha$, $\beta_1$, $\beta_2$, $max\_iter$):<br>
&nbsp; &nbsp; $k$ $\leftarrow$ $0$ <br>
&nbsp; &nbsp; $m$ $\leftarrow$ $0$ <br>
&nbsp; &nbsp; $v$ $\leftarrow$ $0$ <br>
&nbsp; &nbsp; $d$ $\leftarrow$ $\nabla f(x)$ &nbsp; &nbsp; `# Dirección del primer paso` <br>
&nbsp; &nbsp; MIENTRAS $k\leq max\_iter$ and $\lVert d\rVert$ $> 10^{-8}$: <br>
&nbsp; &nbsp; &nbsp; &nbsp; $k\leftarrow k + 1$ <br>
&nbsp; &nbsp; &nbsp; &nbsp; $m\leftarrow \beta_1m + (1-\beta_1)d$ <br>
&nbsp; &nbsp; &nbsp; &nbsp; $v\leftarrow \beta_2v + (1-\beta_2)(d\odot d)$ <br>
&nbsp; &nbsp; &nbsp; &nbsp; $\hat{m}\leftarrow \frac{m}{(1-\beta_1^k)}$ <br>
&nbsp; &nbsp; &nbsp; &nbsp; $\hat{v}\leftarrow \frac{v}{(1-\beta_2^k)}$ <br>
&nbsp; &nbsp; &nbsp; &nbsp; $x$ $\leftarrow$ $x - \alpha \frac{\hat{m}}{\sqrt{\hat{v}}+10^{-8}}$ &nbsp; &nbsp; <br>
&nbsp; &nbsp; &nbsp; &nbsp; $d$ $\leftarrow$ $\nabla f(x)$  <br>
&nbsp; &nbsp; DEVOLVER $x$

## Ejercicios

1. Implementar `adam_cuad` que aplica ADAM a funciones cuadráticas.

In [25]:
def adam_cuad(A, b, x, alpha, beta_1=0.9, beta_2=0.99, eps=1e-5, k_max=1000):
  """
  Aplica ADAM para funciones cuadraticas.
  A : matriz de la funcion cuadratica (np.array)
  b : vector de la funcion cuadratica (np.array)
  x: punto inicial (numpy.array)
  alpha: longitud del paso (float)
  beta_1: parametro del momento (float)
  beta_2: parametro del modificador de la longitud de paso (float)
  eps: valor de tolerancia para la norma del gradiente (float)
  k_max: limite de iteraciones (int)
  """
  # COMPLETAR
  print(f'Cantidad de iteraciones: {k}')
  return x

2. Para corroborar la correcta implementación, aplicar ADAM a la función $f(x)=\frac{1}{2}x^TQx$ para $Q$ definida en el ejercicio anterior, con $N=10$ y $\alpha=0.01$.

In [26]:
N = 10
b = np.zeros(N)
x_0 = np.ones(N)
Q = np.diag(np.arange(N)+1)

x = adam_cuad(???)
x@Q@x

SyntaxError: invalid syntax (42446363.py, line 6)

3. Para $N=1500$ y `k_max=10000`, comparar ADAM ($\alpha=0.01$) con Polyak y Nesterov. Experimentar con distintos puntos iniciales y valores de $\alpha$.

In [ ]:
N = 1500
b = np.zeros(N)
x_0 = np.ones(N)
Q = np.diag(np.arange(N)+1)

adam_start = perf_counter()
x = adam_cuad(Q, b, x_0, alpha=0.01, k_max=10000)
print(f'Tiempo de ADAM: {perf_counter()-adam_start:.2f}')

polyak_start = perf_counter()
x = metodo_polyak_cuad(Q, b, x_0, 1, N, k_max=10000)
print(f'Tiempo del Método de Polyak: {perf_counter()-polyak_start:.2f}')

nesterov_start = perf_counter()
x = metodo_nesterov_cuad(Q, b, x_0, 1, N, k_max=10000)
print(f'Tiempo del Método de Nesterov: {perf_counter()-nesterov_start:.2f}')

Cantidad de iteraciones: 340
Tiempo del Método del Gradiente con mitad de t*: 3.07
Cantidad de iteraciones: 497
Tiempo del Método de Polyak: 4.83
Cantidad de iteraciones: 546
Tiempo del Método de Nesterov: 6.85
